# MVP-L: 3D 객체 추적

이 노트북은 3D 바운딩 박스 기반 객체 추적을 수행합니다 (Kalman Filter + AB3DMOT).

## 목표
- Kalman Filter 기반 추적
- AB3DMOT 또는 SimpleTrack 구현
- 3D 궤적 시각화

In [ ]:
# Cell 1: 라이브러리 임포트
import numpy as np
from scipy.spatial.distance import cdist
from filterpy.kalman import KalmanFilter
from collections import defaultdict
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# filterpy 설치: pip install filterpy
print("라이브러리 로드 완료")

In [ ]:
# Cell 2: 3D Kalman Filter 기반 트랙 클래스
class Track3D:
    """3D 객체 트랙"""
    def __init__(self, bbox_3d, track_id, frame_id):
        """
        Args:
            bbox_3d: dict with keys 'center', 'size', 'rotation'
            track_id: 트랙 ID
            frame_id: 프레임 ID
        """
        self.track_id = track_id
        self.frame_ids = [frame_id]
        self.bboxes = [bbox_3d]
        self.time_since_update = 0
        self.hit_streak = 1
        
        # Kalman Filter 초기화 (상태: [x, y, z, vx, vy, vz])
        self.kf = KalmanFilter(dim_x=9, dim_z=3)  # 상태: [x, y, z, vx, vy, vz, ax, ay, az]
        
        # 상태 변수: [x, y, z, vx, vy, vz, ax, ay, az]
        self.kf.x = np.array([
            bbox_3d['center'][0], bbox_3d['center'][1], bbox_3d['center'][2],
            0, 0, 0, 0, 0, 0  # 초기 속도, 가속도는 0
        ])
        
        # 상태 전이 행렬 (constant velocity model)
        dt = 1.0  # 프레임 간 시간 간격
        self.kf.F = np.array([
            [1, 0, 0, dt, 0, 0, 0.5*dt**2, 0, 0],
            [0, 1, 0, 0, dt, 0, 0, 0.5*dt**2, 0],
            [0, 0, 1, 0, 0, dt, 0, 0, 0.5*dt**2],
            [0, 0, 0, 1, 0, 0, dt, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, dt, 0],
            [0, 0, 0, 0, 0, 1, 0, 0, dt],
            [0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 1]
        ])
        
        # 측정 행렬 (위치만 측정)
        self.kf.H = np.array([
            [1, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0]
        ])
        
        # 공분산 행렬
        self.kf.P *= 1000
        self.kf.R = np.eye(3) * 1.0  # 측정 노이즈
        self.kf.Q = np.eye(9) * 0.1  # 프로세스 노이즈
    
    def predict(self):
        """다음 프레임 예측"""
        self.kf.predict()
        return self.kf.x[:3]  # 위치 반환
    
    def update(self, bbox_3d):
        """측정값으로 업데이트"""
        measurement = np.array(bbox_3d['center'])
        self.kf.update(measurement)
        self.bboxes.append(bbox_3d)
        self.time_since_update = 0
        self.hit_streak += 1
    
    def get_state(self):
        """현재 상태 반환"""
        state = self.kf.x[:3]
        return {
            'center': state,
            'size': self.bboxes[-1]['size'],
            'rotation': self.bboxes[-1].get('rotation', 0)
        }


class Tracker3D:
    """3D 객체 추적기"""
    def __init__(self, max_age=30, min_hits=3, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.tracks = []
        self.frame_count = 0
        self.next_id = 1
    
    def update(self, detections_3d):
        """
        새로운 3D 탐지로 추적 업데이트
        
        Args:
            detections_3d: list of dict with keys 'center', 'size', 'rotation'
        """
        self.frame_count += 1
        
        # 예측
        predicted_states = []
        for track in self.tracks:
            pred_state = track.predict()
            predicted_states.append(pred_state)
        
        # 매칭 (거리 기반)
        if len(self.tracks) > 0 and len(detections_3d) > 0:
            # 거리 행렬 계산
            pred_centers = np.array([s for s in predicted_states])
            det_centers = np.array([d['center'] for d in detections_3d])
            distances = cdist(pred_centers, det_centers)
            
            # 간단한 greedy matching
            matched = []
            unmatched_tracks = list(range(len(self.tracks)))
            unmatched_detections = list(range(len(detections_3d)))
            
            while True:
                if len(unmatched_tracks) == 0 or len(unmatched_detections) == 0:
                    break
                
                min_dist = np.inf
                best_match = None
                
                for t in unmatched_tracks:
                    for d in unmatched_detections:
                        if distances[t, d] < min_dist and distances[t, d] < 2.0:  # 2m 임계값
                            min_dist = distances[t, d]
                            best_match = (t, d)
                
                if best_match is None:
                    break
                
                matched.append(best_match)
                unmatched_tracks.remove(best_match[0])
                unmatched_detections.remove(best_match[1])
        else:
            matched = []
            unmatched_tracks = list(range(len(self.tracks))) if len(self.tracks) > 0 else []
            unmatched_detections = list(range(len(detections_3d)))
        
        # 매칭된 트랙 업데이트
        for track_idx, det_idx in matched:
            self.tracks[track_idx].update(detections_3d[det_idx])
        
        # 새 트랙 생성
        for det_idx in unmatched_detections:
            track = Track3D(detections_3d[det_idx], self.next_id, self.frame_count)
            self.tracks.append(track)
            self.next_id += 1
        
        # 오래된 트랙 제거
        self.tracks = [t for t in self.tracks if t.time_since_update < self.max_age]
        
        return self.tracks

print("3D 추적기 클래스 정의 완료")

In [ ]:
# Cell 3: 3D 궤적 시각화
def visualize_3d_trajectories(points, tracks, ax=None):
    """
    3D 궤적 시각화
    """
    if ax is None:
        fig = plt.figure(figsize=(14, 10))
        ax = fig.add_subplot(111, projection='3d')
    
    # 포인트 클라우드 (샘플링)
    sample_indices = np.random.choice(len(points), min(5000, len(points)), replace=False)
    ax.scatter(points[sample_indices, 0], 
              points[sample_indices, 1], 
              points[sample_indices, 2], 
              c='gray', s=0.1, alpha=0.3)
    
    # 각 트랙의 궤적 그리기
    colors = plt.cm.tab20(np.linspace(0, 1, len(tracks)))
    for i, track in enumerate(tracks):
        if len(track.bboxes) < 2:
            continue
        
        # 궤적 추출
        trajectory = np.array([bbox['center'] for bbox in track.bboxes])
        
        # 궤적 선 그리기
        ax.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2],
               color=colors[i % len(colors)], linewidth=2, label=f'Track {track.track_id}')
        
        # 현재 위치 표시
        current_pos = trajectory[-1]
        ax.scatter(current_pos[0], current_pos[1], current_pos[2],
                  color=colors[i % len(colors)], s=100, marker='o')
    
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title('3D Object Tracking Trajectories')
    ax.legend()
    
    plt.show()
    return ax

print("궤적 시각화 함수 정의 완료")